# Food Recommendation Model Overview

This code implements a neural network model using PyTorch for recommending food items based on user-specific inputs such as age, weight, and activity level. The model processes various features like the physical dimensions of food images (width, height) and user characteristics to predict the best-suited food class from a predefined set of 12 food items. The main components of the code are as follows:

### Data Preparation

1. **Food Classes**: 
    - A list of 12 food items such as "nasi", "ayam", "ikan", etc., each associated with nutritional information (calories, protein, fat).

2. **Nutritional Values**:
    - Dummy nutritional values for each food class are provided, defining calories, protein, and fat per 100 grams of food.

3. **User Data**:
    - User-specific attributes such as age, weight, and activity levels are defined as input features to the model.

4. **Dataset Generation**:
    - A dummy dataset is created with random values for food dimensions, user features, and calculated nutritional values for training the model.

### Model Architecture

The `FoodRecommendationModel` is a feedforward neural network built with the following layers:
- **Input Layer**: The input size is set to 7, representing the features: food image dimensions (width, height), user-specific data (age, weight, activity level), and the nutritional information (calories, protein, fat).
- **Two Hidden Layers**: Each hidden layer contains 128 neurons with ReLU activation functions.
- **Output Layer**: This layer produces a probability distribution across the 12 food classes, indicating the recommended food based on the input.

### Model Training

- **Loss Function**: The model uses `CrossEntropyLoss`, suitable for multi-class classification tasks.
- **Optimizer**: Adam optimizer with a learning rate of 0.001 is used to update the model's parameters during training.
- **Training Loop**: A dummy training loop is run for 10 epochs with random data to simulate the training process.

### Example Prediction

After training, the model can predict the recommended food class based on the given input features. For instance, providing inputs such as:
- Food dimensions: `250 x 300` 
- Food weight: `150g`
- User details: age `36`, weight `75kg`, and activity level `1.75`
The model outputs a recommendation for the most suitable food from the available classes.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

# Dummy data food classes
food_classes = ["nasi", "ayam", "kentang", "ikan", "sayur", "buah", "telur", "daging", "susu", "roti", "keju", "mie"]
num_classes = len(food_classes)

# Dummy nutritional values for each food class
nutritional_values = {
    "nasi": {"kalori": 130, "protein": 2.5, "lemak": 0.3},
    "ayam": {"kalori": 239, "protein": 27, "lemak": 14},
    "kentang": {"kalori": 77, "protein": 2, "lemak": 0.1},
    "ikan": {"kalori": 206, "protein": 22, "lemak": 12},
    "sayur": {"kalori": 25, "protein": 1.8, "lemak": 0.2},
    "buah": {"kalori": 52, "protein": 0.5, "lemak": 0.2},
    "telur": {"kalori": 155, "protein": 13, "lemak": 11},
    "daging": {"kalori": 250, "protein": 26, "lemak": 15},
    "susu": {"kalori": 42, "protein": 3.4, "lemak": 1},
    "roti": {"kalori": 265, "protein": 9, "lemak": 3.2},
    "keju": {"kalori": 402, "protein": 25, "lemak": 33},
    "mie": {"kalori": 138, "protein": 4.5, "lemak": 2}
}

# Dummy user data
user_data = {
    "user_age": [25, 30, 36, 40, 50],
    "user_weight": [70, 80, 90, 60, 75],
    "activity_level": [1.2, 1.5, 1.75, 2.0]  # This represents activity multipliers (sedentary, moderate, active, very active)
}

# Creating dummy dataset for training
def create_dummy_dataset(num_samples=100):
    data = []
    for _ in range(num_samples):
        width = np.random.randint(50, 500)  # Dummy width
        height = np.random.randint(50, 500)  # Dummy height
        food_class_idx = np.random.randint(0, num_classes)  # Random food class
        food_class = food_classes[food_class_idx]
        food_weight = np.random.uniform(50, 300)  # Random food weight
        user_age = np.random.choice(user_data["user_age"])
        user_weight = np.random.choice(user_data["user_weight"])
        activity_level = np.random.choice(user_data["activity_level"])

        # Get nutritional data based on the food class and weight
        nutrisi = nutritional_values[food_class]
        kalori = (nutrisi["kalori"] / 100) * food_weight
        protein = (nutrisi["protein"] / 100) * food_weight
        lemak = (nutrisi["lemak"] / 100) * food_weight

        # Store the data
        data.append([width, height, food_class_idx, food_weight, user_age, user_weight, activity_level, kalori, protein, lemak])

    columns = ['width', 'height', 'food_class_idx', 'food_weight', 'user_age', 'user_weight', 'activity_level', 'kalori', 'protein', 'lemak']
    return pd.DataFrame(data, columns=columns)

# Generate dataset
dataset = create_dummy_dataset(num_samples=500)
print(dataset.head())

# Save dataset for further use
dataset.to_csv('food_recommendation_dummy_data.csv', index=False)

   width  height  food_class_idx  food_weight  user_age  user_weight  \
0    244     112               4   145.127506        50           70   
1    446     319              10   220.660966        36           80   
2    451     204               1    51.072504        36           60   
3    122     298              11   235.484485        25           90   
4    236     233               4   271.326256        40           80   

   activity_level      kalori    protein      lemak  
0             1.5   36.281877   2.612295   0.290255  
1             2.0  887.057085  55.165242  72.818119  
2             2.0  122.063283  13.789576   7.150150  
3             1.2  324.968589  10.596802   4.709690  
4             1.2   67.831564   4.883873   0.542653  


In [2]:
class FoodRecommendationModel(nn.Module):
    def __init__(self, num_classes, input_size=7, hidden_size=128):
        super(FoodRecommendationModel, self).__init__()
        # Input size corresponds to width, height, user features, etc.
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)  # Output for food classes (rekomendasi makanan)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No activation since it's multi-class prediction
        return x

# Hyperparameters
input_size = 7  # Features: width, height, user_age, user_weight, activity_level, etc.
hidden_size = 128
num_classes = len(food_classes)

# Initialize model
model = FoodRecommendationModel(num_classes=num_classes, input_size=input_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Multi-class classification task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy training loop (without actual data loader for now)
num_epochs = 10
for epoch in range(num_epochs):
    # Dummy input, let's assume we have a batch of size 64
    inputs = torch.randn(64, input_size)
    labels = torch.randint(0, num_classes, (64,))  # Random food class labels
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [2/10], Loss: 2.4868
Epoch [4/10], Loss: 2.4867
Epoch [6/10], Loss: 2.4737
Epoch [8/10], Loss: 2.4655
Epoch [10/10], Loss: 2.4982


In [5]:
# Example prediction with all 7 features
user_input = torch.tensor([[250, 300, 5, 150, 36, 75, 1.75]])  # Input: width, height, food_class_idx, food_weight, user_age, user_weight, activity_level
output = model(user_input)
predicted_food_class = torch.argmax(output, dim=1)
print(f"Rekomendasi makanan: {food_classes[predicted_food_class.item()]}")


Rekomendasi makanan: roti
